**1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,
fecha). Se pide resolver utilizando PySpark:**
- A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
- B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
- C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido)

In [1]:
data = sc.textFile("ej1.csv").map(lambda x: x.split(','))
data.take(10)

[['10042', 'Quimica', '5', '02/17/2018'],
 ['10074', 'Algo1', '1', '12/15/2017'],
 ['10099', 'Analisis II', '2', '12/02/2018'],
 ['10024', 'Algo2', '7', '08/15/2017'],
 ['10098', 'Taller de programacion', '6', '05/20/2017'],
 ['10015', 'Analisis II', '5', '09/09/2018'],
 ['10037', 'Quimica', '1', '04/21/2019'],
 ['10013', 'Quimica', '10', '05/29/2019'],
 ['10062', 'Analisis II', '9', '04/12/2017'],
 ['10060', 'Algo3', '4', '10/11/2017']]

In [2]:
aprobadosUltimoAnio = data.filter(lambda x: (int(x[2])>=4) & (x[3]>"01/01/2017") )
cant_aprobados = aprobadosUltimoAnio.map(lambda x: x[0]).distinct().count()
print("El ultimo anio, aprobaron %d alumnos"%cant_aprobados)

El ultimo anio, aprobaron 101 alumnos


In [3]:
porAlumno = data.map(lambda x: (x[0],(int(x[2]),1)))
promPorAlumno = porAlumno.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).map(lambda x:(x[0],x[1][0]/x[1][1]))
promPorAlumno.take(5)

[('10042', 4.7272727272727275),
 ('10099', 4.090909090909091),
 ('10098', 5.076923076923077),
 ('10015', 5.25),
 ('10037', 5.133333333333334)]

In [4]:
nombres = sc.textFile("ej1_nombres.csv").map(lambda x: x.split(','))
joined = promPorAlumno.join(nombres)
mayorPromedio = joined.reduce(lambda x,y: x if x[1]>y[1] else y)
print("El nombre del alumno con mejor promedio es %s, con promedio %.2f "%(mayorPromedio[1][1],mayorPromedio[1][0]))

El nombre del alumno con mejor promedio es Mirella Rosten, con promedio 8.50 


2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:

In [5]:
ventas = sc.textFile('ej2.csv').map(lambda x: x.split(','))
detalle = sc.textFile('ej2_productos.csv').map(lambda x: x.split(','))
ventas.take(5)

[['11/04/2019', '20', '338'],
 ['12/09/2019', '1', '412'],
 ['07/10/2019', '61', '395'],
 ['09/26/2019', '94', '52'],
 ['11/03/2019', '88', '216']]

A. Cuál es el producto más vendido.

In [6]:
productos = ventas.map(lambda x: (x[1],1) )
porProducto = productos.reduceByKey(lambda x,y:x+y)
joined = porProducto.join(detalle)
joined.take(5)

[('20', (7, 'neque aenean auctor gravida sem praesent id massa')),
 ('88', (12, 'enim in tempor turpis nec euismod scelerisque quam')),
 ('77',
  (15, 'est lacinia nisi venenatis tristique fusce congue diam id ornare')),
 ('10', (9, 'aliquam non mauris morbi non lectus aliquam sit amet diam')),
 ('40', (13, 'laoreet ut rhoncus'))]

In [7]:
prodMasVendido = joined.reduce(lambda x,y: x if x[1][0] > y[1][0] else y)
print("El producto mas vendido es %s, con %d unidades"%(prodMasVendido[1][1],prodMasVendido[1][0]))

El producto mas vendido es nam congue risus semper porta volutpat quam pede, con 17 unidades


B. Cuál es la categoría de productos más vendida.

In [8]:
porCategoria = detalle.map(lambda x: (x[2],1)).reduceByKey(lambda x,y:x+y)
maxCategoria = porCategoria.reduce(lambda x,y: x if x[1]>y[1] else y)
print("La categoria mas vendida es %s"%maxCategoria[0])

La categoria mas vendida es Computacion


C. Cuál es el top5 de productos más vendidos generando un RDD con (código de
producto, descripción, cantidad de ventas)

In [9]:
descripciones = detalle.map(lambda x: (x[0],x[1]))
joined_prod_desc = porProducto.join(detalle)
joined_prod_desc.take(5)
top5prod = joined_prod_desc.map(lambda x: (x[0],x[1][1],x[1][0])).takeOrdered(5, lambda x: -x[2])
top5RDD = sc.parallelize(top5prod)
top5RDD.take(10)

[('71', 'nam congue risus semper porta volutpat quam pede', 17),
 ('65', 'id pretium iaculis diam', 16),
 ('77',
  'est lacinia nisi venenatis tristique fusce congue diam id ornare',
  15),
 ('80',
  'mauris viverra diam vitae quam suspendisse potenti nullam porttitor lacus',
  15),
 ('15', 'ante ipsum primis in faucibus', 15)]

D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando
para este análisis solo los productos que cuenten con al menos 50 ventas en el último
año.

In [10]:
from datetime import datetime as dt
productosVendidosUltAnio = ventas.filter(lambda x: dt.strptime(x[0],'%m/%d/%Y') > dt.strptime('01/01/2019','%d/%m/%Y'))
productosConCantVentasOk = productosVendidosUltAnio.map(lambda x: (x[1],1)).reduceByKey(lambda x,y:x+y).filter(lambda x: x[1]>=10)
productosConListaDePrecios = productosVendidosUltAnio.map(lambda x: (x[1],int(x[2]))).groupByKey()
joined = (productosConCantVentasOk.join(productosConListaDePrecios).map(lambda x: (x[0],list(x[1][1])))).join(detalle)
productoMaxAumento = joined.map(lambda x: (x[0],max(x[1][0])/min(x[1][0]),x[1][1])).reduce(lambda x,y: x if x[1]>y[1] else y)
print("El producto con maximo aumento es %s, con un aumento proporcional de %.2f"%(productoMaxAumento[2],productoMaxAumento[1]-1))

El producto con maximo aumento es erat nulla tempus vivamus in felis eu sapien, con un aumento proporcional de 8.82


4- Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide
programar en PySpark un programa que encuentre el rectángulo de superficie mínima que
contiene al punto (w,z)

In [39]:
rdd = sc.textFile('ej4.csv').map(lambda x: tuple(int(n) for n in x.split(',')) )
rdd.take(5)

[(87, 11, 52, 40),
 (81, 93, 23, 4),
 (58, 52, 85, 11),
 (87, 76, 19, 54),
 (37, 38, 70, 59)]

In [51]:
def contiene_punto(tupla):
    x1,x2,y1,y2 = tupla[0]
    w,z = tupla[1]
    return (w >= x1 and w <= x2) and (z >= y1 and z <= y2)

def area(tupla):
    x1,x2,y1,y2 = tupla[0]
    return (x2-x1)*(y2-y1)

punto = (88,17)
rdd_a = rdd.map(lambda x: (x,punto))
rdd_filtrado = rdd_a.filter(contiene_punto)
rdd_area = rdd_filtrado.map(area)
area_minima = rdd_area.reduce(lambda x,y: x if x<y else y)
print("El area minima de los rectangulos que contienen al %s es %d "%((punto),area_minima))

El area minima de los rectangulos que contienen al (88, 17) es 528 


6- UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus
vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los
siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD),
distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [88]:
from datetime import datetime as dt
dataUber = sc.textFile("ej6.csv").map(lambda x: x.split(','))
dataUber.take(5)
fechaLimiteInf = dt.strptime("2016-04-01",'%Y-%m-%d')
fechaLimiteSup = dt.strptime("2016-04-30",'%Y-%m-%d')
dataAbril2016 = dataUber.filter(lambda x: dt.strptime(x[4],'%Y-%m-%d') >= fechaLimiteInf and dt.strptime(x[4],'%Y-%m-%d') <= fechaLimiteSup)
dataPorConductor = dataAbril2016.map(lambda x: (x[0],int(x[5]))).groupByKey()
dataConPromedio = dataPorConductor.map(lambda x: (x[0],sum(list(x[1]))/len(x[1])))
mejorPromedio = dataConPromedio.reduce(lambda x,y: x if x[1]>y[1] else y)
print("El conductor con mejor promedio en Abril de 2016 es el %s con %.2f kilometros promedio por viaje"%(mejorPromedio[0],mejorPromedio[1]))

El conductor con mejor promedio es el 10528 con 3571.75 kilometros promedio por viaje
